In [ ]:
# Non Layers Version

import numpy as np
import rhino3dm

# Function to extract geometry objects from a Rhino3dm model.
def extract_geometry(model):
    """
    Extracts geometry objects from a Rhino3dm model.
    Returns a list of geometry objects (e.g., surfaces, Breps, Extrusions).
    """
    geometry = []
    for obj in model.Objects:
        geom = obj.Geometry
        if geom:
            geometry.append(geom)
    return geometry

# Function to compute the normal vector for a given surface.
def get_normal_vector(surface):
    """
    Computes the normal vector for a given surface.
    For Extrusion objects, converts them to a Brep (calling ToBrep with a boolean argument)
    and uses the normal of the first face.
    For Brep objects, uses the normal of the first face.
    If the object has a NormalAt method, it will be used as a fallback.
    """
    try:
        type_name = type(surface).__name__
        if type_name == "Extrusion":
            # Convert the Extrusion to a Brep (providing False for splitKinkyFaces)
            brep = surface.ToBrep(False)
            if brep and len(brep.Faces) > 0:
                face = brep.Faces[0]
                normal = face.NormalAt(0.5, 0.5)
                norm_val = np.linalg.norm([normal.X, normal.Y, normal.Z])
                if norm_val != 0:
                    return np.array([normal.X, normal.Y, normal.Z]) / norm_val
        elif type_name == "Brep":
            if len(surface.Faces) > 0:
                face = surface.Faces[0]
                normal = face.NormalAt(0.5, 0.5)
                norm_val = np.linalg.norm([normal.X, normal.Y, normal.Z])
                if norm_val != 0:
                    return np.array([normal.X, normal.Y, normal.Z]) / norm_val
        # Fallback: if the surface has a NormalAt method directly.
        if hasattr(surface, "NormalAt"):
            normal = surface.NormalAt(0.5, 0.5)
            norm_val = np.linalg.norm([normal.X, normal.Y, normal.Z])
            if norm_val != 0:
                return np.array([normal.X, normal.Y, normal.Z]) / norm_val
    except Exception as e:
        print("Error computing normal for surface:", e)
    # Fallback normal (horizontal)
    return np.array([0, 0, 1])

# Function to classify geometry solely based on the computed normals.
def classify_geometry(geometry):
    classified = {"walls": [], "floors": [], "ceilings": [], "columns": [], "beams": []}
    
    for geom in geometry:
        normal = get_normal_vector(geom)
        # If the normal is nearly vertical (i.e., [0, 0, ±1]), classify as a floor.
        if np.allclose(np.abs(normal), [0, 0, 1], atol=0.2):
            classified["floors"].append(geom)
        # If the normal is nearly horizontal (i.e., [±1, 0, 0] or [0, ±1, 0]), classify as a wall.
        elif (np.allclose(np.abs(normal), [1, 0, 0], atol=0.2) or 
              np.allclose(np.abs(normal), [0, 1, 0], atol=0.2)):
            classified["walls"].append(geom)
        else:
            # Fallback: if the normal doesn't clearly indicate either, treat as floor.
            classified["floors"].append(geom)
        
        # Placeholders for additional classification for columns and beams.
        if hasattr(geom, 'IsCylinder') and geom.IsCylinder:
            classified["columns"].append(geom)
        if hasattr(geom, 'IsBeam') and geom.IsBeam:
            classified["beams"].append(geom)
            
    return classified

# Example usage for Stage 1:
file_path = "basicformMETERS.3dm"  # Replace with the actual path to your Rhino file.
try:
    model = rhino3dm.File3dm.Read(file_path)
    geometry = extract_geometry(model)
    classified_elements = classify_geometry(geometry)
    print("Geometry Classification Results:")
    for key, items in classified_elements.items():
        print(f"  {key.capitalize()}: {len(items)} element(s)")
except Exception as e:
    print("Error loading or processing the file:", e)
